In [ ]:
import os
from dotenv import load_dotenv
load_dotenv(override=True)


In [ ]:
import os
from clickhouse_driver import Client
service_uri=os.environ.get('CH_SERVICE_URI')

client = Client(service_uri.split(':')[0],
                user=os.environ.get('CH_USER'),
                password=os.environ.get('CH_PASSWORD'),
                secure=True,
                port=service_uri.split(':')[1]
                )

client.execute('SHOW DATABASES')

In [ ]:
first_query = 'select lat, long from default.bus_positions where (spd > 15.0) limit 500'
out = client.execute(first_query)
len(out)

In [ ]:
count_query = 'select count() from default.bus_positions'
client.execute(count_query)

In [ ]:
next_query = 'select lat, long from default.bus_positions where (spd > 20.0) limit 500'
out = client.execute(next_query)
len(out)

In [ ]:
import folium
m = folium.Map(location=[60.40, 25.1067], zoom_start=11)
for point in out:
    try:
        folium.Marker(point).add_to(m)
    except:
        pass
    
m

In [ ]:
marker_colors = ['red', 'darkred',  'lightred', 'orange', 'beige',
                 'green', 'darkgreen', 'lightgreen',
                 'blue', 'darkblue', 'cadetblue', 'lightblue',
                 'purple',  'darkpurple', 'pink',
                 'white', 'gray', 'lightgray', 'black']


                 

In [ ]:
out = client.execute('select max (spd) from default.bus_positions')
out


This is an interesting one as it lets us map colors to speed :)


In [ ]:
import math
def colorise(speed_in: float)-> str:
    return marker_colors[math.floor(speed_in/50)]


In [ ]:
import folium
def make_map(my_points):
    map = folium.Map(location=[60.40, 25.1067], zoom_start=11)

    for point in my_points:
        try:
            folium.Marker(location=[point[0], point[1]],  icon=folium.Icon(color=colorise(point[2])) ).add_to(map)
        except:
            pass

    return map

In [ ]:
# aggregate points to polyline sorted by the timestamp
import folium
track_query = (
'select oper, veh, '
'groupArray(tuple(tsi, tuple(lat, long))) as track, '
'arraySort((x) -> x.1, track) '
'from default.bus_positions '
'group by oper, veh')
result = client.execute(track_query)


In [ ]:
# create polyline for the first vehicle
# TODO 1) detect discontinuation based on time between points 2) still some strange overlapping if using operator and vehicle id as unique vehicle indentifier
map = folium.Map(location=[60.40, 25.1067], zoom_start=11)
color_str =''
for vehicle in result:
    points = [x[1] for x in vehicle[3]]
    folium.PolyLine(points).add_to(map)

#
# for point in out:
#    try:
#        folium.Marker(location=[point[0], point[1]], color = colorise(point[2])).add_to(map)
#
#    except:
#        pass
        
map